In [136]:
import math as m
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from scipy.special import erf
%matplotlib inline

basis = np.array([[13.00773, 1.962079, 0.444529, 0.1219492, 13.00773, 1.962079, 0.444529, 0.1219492],
                  [0       , 0       , 0       , 0        , 1       , 1       , 1       , 1        ]])

In [137]:
def F(t) :
    if t !=0 :
        out = np.sqrt(np.pi)/2*1/np.sqrt(t)*erf(np.sqrt(t))
        return out
    else :
        return 1
def S(basis) :
    N = basis.shape[1]
    Overlap_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            a = basis[0,p]
            b = basis[0,q]
            Ra = basis[1,p]
            Rb = basis[1,q]
            Overlap_matrix[p,q] = ( np.pi/(a+b) )**(3/2)*np.exp( -a*b/(a+b)*abs(Ra-Rb)**2 )
            Overlap_matrix[q,p] = Overlap_matrix[p,q]
    return Overlap_matrix

def T(basis) :
    N = basis.shape[1]
    Kintetic_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            a = basis[0,p]
            b = basis[0,q]
            Ra = basis[1,p]
            Rb = basis[1,q]
            Kintetic_matrix[p,q] = 1/2*a*b/(a+b)*( 6 - 4*a*b/(a+b)*abs(Ra-Rb)**2 )*( np.pi/(a+b) )**(1.5)*np.exp( -a*b/(a+b)*abs(Ra-Rb)**2 )
            Kintetic_matrix[q,p] = Kintetic_matrix[p,q]
    return Kintetic_matrix

def A(basis) :
    N = basis.shape[1]
    Coulomb_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            a = basis[0,p]
            b = basis[0,q]
            Ra = basis[1,p]
            Rb = basis[1,q]
            Rp = ( a*Ra + b*Rb )/( a + b )
            Coulomb_matrix[p,q] = -2*np.pi/( a + b )*np.exp( -a*b/( a + b)*abs(Ra - Rb)**2)*( F( (a+b)*abs(Rp - 1)**2) + F( (a+b)*abs(Rp - 0)**2) )
            Coulomb_matrix[q,p] = Coulomb_matrix[p,q]
    return Coulomb_matrix

def Q(basis) : 
    N = basis.shape[1]
    Electron_matrix = np.zeros((N,N,N,N))
    Qt = np.zeros((N,N,N,N)) 
    for p in range(N) :
        for q in range(N) :
            for r in range(N) :
                for s in range(N) :
                        a = basis[0,p]
                        b = basis[0,r]
                        c = basis[0,q]
                        d = basis[0,s]
                        Ra = basis[1,p]
                        Rb = basis[1,r]
                        Rc = basis[1,q]
                        Rd = basis[1,s]
                        Rp = ( a*Ra + c*Rc )/( a + c )
                        Rq = ( b*Rb + d*Rd )/( b + d )
                        Electron_matrix[p,r,q,s] = 2*np.pi**(5/2)/(a + c)/(b + d)/(a + b + c + d)**(1/2)*np.exp( -a*c/(a + c)*abs( Ra - Rc )**2 -b*d/(b + d)*abs( Rb - Rd )**2 )*F( (a + c)*(b + d)/(a + b + c + d)*abs(Rp - Rq)**2 )
    for p in range(N) :
        for q in range(N) :
            for r in range(N) : 
                for s in range(N) :
                    Qt[p,r,q,s] = Electron_matrix[p,r,q,s]
    return Qt

def normalize(C,S) :
    I = 0
    N = np.shape(C)[0]
    for r in range(N) :
        for s in range(N) : 
            I = I + S[r,s]*C[r]*C[s]
    C = C/np.sqrt(I)
    return C
Q(basis)

array([[[[7.16656272e-03, 1.40327573e-02, 1.59131785e-02, ...,
          2.41872984e-03, 1.03200091e-02, 1.45044484e-02],
         [1.40327573e-02, 2.85331048e-02, 3.25885743e-02, ...,
          4.97299022e-03, 2.11480114e-02, 2.97527970e-02],
         [1.59131785e-02, 3.25885743e-02, 3.72736248e-02, ...,
          5.69239533e-03, 2.41914812e-02, 3.40415318e-02],
         ...,
         [2.41872984e-03, 4.97299022e-03, 5.69239533e-03, ...,
          9.42935629e-04, 3.76867205e-03, 5.22842164e-03],
         [1.03200091e-02, 2.11480114e-02, 2.41914812e-02, ...,
          3.76867205e-03, 1.57778464e-02, 2.21244512e-02],
         [1.45044484e-02, 2.97527970e-02, 3.40415318e-02, ...,
          5.22842164e-03, 2.21244512e-02, 3.11038743e-02]],

        [[1.40327573e-02, 6.26330492e-02, 1.04818991e-01, ...,
          1.82202217e-02, 7.11658576e-02, 1.08293414e-01],
         [2.85331048e-02, 1.37018862e-01, 2.32971581e-01, ...,
          4.06674318e-02, 1.58402522e-01, 2.41604481e-01],
        

In [140]:
def H2(basis,C) :
    
    S_matrix = S(basis)
    T_matrix = T(basis)
    A_matrix = A(basis)
    Q_tensor = Q(basis)
    Q_matrix = np.zeros((8,8)) 

    C = normalize(C,S_matrix)
    density_matrix = np.zeros((8,8))

    for p in range(8) :
        for q in range(8) :
            density_matrix[p,q] = 2*C[p]*C[q]
    for r in range(8) : 
        for s in range(8) : 
            Q_matrix = Q_matrix + 1/2*density_matrix[r,s]*(Q_tensor[:,r,:,s] )
    
    eigenenergy, eigenvector= eigh((T_matrix + A_matrix + Q_matrix), S_matrix)
    eigenenergy = eigenenergy[0]
    eigenvector = eigenvector[:,0]
    
    total_energy = 0 
    for p in range(8) :
        for q in range(8) :
            total_energy = total_energy  + (A_matrix[p,q]+T_matrix[p,q])*density_matrix[p,q]
            for r in range(8) :
                for s in range(8) :
                    total_energy = total_energy + 1/4*Q_tensor[p,r,q,s]*density_matrix[p,q]*density_matrix[r,s]
                    
    return np.array([eigenvector]), total_energy, S_matrix, A_matrix, T_matrix, Q_matrix


In [141]:
def SCF(basis) : 
    n = 10
    C = np.array([1,1,1,1,1,1,1,1])
    for ele in range(n) :
        total_energy = H2(basis,C)[1]
        C = H2(basis,C)[0][0,:]
        print(total_energy+1)
    return total_energy+1, C

SCF(basis)

-0.811798240189592
-1.0609067656284896
-1.077332543531524
-1.078461697260797
-1.0785415597365957
-1.078547182318042
-1.0785475787340828
-1.0785476066728972
-1.0785476086421482
-1.0785476087809456


(-1.0785476087809456,
 array([0.09256155, 0.16517976, 0.12012284, 0.02115452, 0.09256155,
        0.16517976, 0.12012284, 0.02115452]))